In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .config("jsonstore.rdd.partitions", 1)\
    .getOrCreate()

In [4]:

df=spark.read.load('shake_classification', "com.cloudant.spark")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()

+-----+--------+-----+-----+-----+--------------------+--------------------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+--------+-----+-----+-----+--------------------+--------------------+
|    0|asdfghjk|  0.0|  0.0|  0.0|001fd4da2a1ced55e...|1-461daac6b553896...|
|    0|asdfghjk|-0.02|-0.02|-0.02|001fd4da2a1ced55e...|1-fbc3c86a654a820...|
|    0|asdfghjk|  0.0|  0.0|  0.0|001fd4da2a1ced55e...|1-461daac6b553896...|
|    0|asdfghjk|-0.02|-0.02|-0.02|001fd4da2a1ced55e...|1-fbc3c86a654a820...|
|    0|asdfghjk| 0.02| 0.02| 0.02|001fd4da2a1ced55e...|1-5c4226e5c41f324...|
|    0|asdfghjk|-0.01|-0.01|-0.01|001fd4da2a1ced55e...|1-c1f52a49af5db18...|
|    0|asdfghjk|  0.0|  0.0|  0.0|001fd4da2a1ced55e...|1-461daac6b553896...|
|    0|asdfghjk| 0.01| 0.01| 0.01|001fd4da2a1ced55e...|1-a4b6e8d60269136...|
|    0|asdfghjk| 0.01| 0.01| 0.01|001fd4da2a1ced55e...|1-a4b6e8d60269136...|
|    0|asdfghjk|-0.01|-0.01|-0.01|001fd4da2a1ced55e...|1-c1f52a49af5db18...|

In [5]:
spark.sql("SELECT count(class), class from df group by class").show()

+------------+-----+
|count(class)|class|
+------------+-----+
|         436|    0|
|         600|    1|
+------------+-----+



In [6]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X","Y","Z"],
                                  outputCol="features")

In [7]:
from pyspark.ml.clustering import KMeans

clust = KMeans().setK(2).setSeed(1)

In [8]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, clust])
model = pipeline.fit(df)

In [9]:
prediction = model.transform(df)
prediction.show()

+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|           features|prediction|
+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|    0|asdfghjk|  0.0|  0.0|  0.0|001fd4da2a1ced55e...|1-461daac6b553896...|          (3,[],[])|         1|
|    0|asdfghjk|-0.02|-0.02|-0.02|001fd4da2a1ced55e...|1-fbc3c86a654a820...|[-0.02,-0.02,-0.02]|         1|
|    0|asdfghjk|  0.0|  0.0|  0.0|001fd4da2a1ced55e...|1-461daac6b553896...|          (3,[],[])|         1|
|    0|asdfghjk|-0.02|-0.02|-0.02|001fd4da2a1ced55e...|1-fbc3c86a654a820...|[-0.02,-0.02,-0.02]|         1|
|    0|asdfghjk| 0.02| 0.02| 0.02|001fd4da2a1ced55e...|1-5c4226e5c41f324...|   [0.02,0.02,0.02]|         1|
|    0|asdfghjk|-0.01|-0.01|-0.01|001fd4da2a1ced55e...|1-c1f52a49af5db18...|[-0.01,-0.01,-0.01]|         1|
|    0|asdfghjk|  0.0|  0.0|

In [10]:
prediction.createOrReplaceTempView('prediction')
spark.sql('''
select max(correct)/max(total) as accuracy from (

    select sum(correct) as correct, count(correct) as total from (
        select case when class != prediction then 1 else 0 end as correct from prediction 
    ) 
    
    union
    
    select sum(correct) as correct, count(correct) as total from (
        select case when class = prediction then 1 else 0 end as correct from prediction 
    ) 
)
''').rdd.map(lambda row: row.accuracy).collect()[0]

0.6032818532818532

In [11]:
!rm -Rf a2_m3.json

In [12]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2019-01-12 03:54:36--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2019-01-12 03:54:36 (21.1 MB/s) - ‘rklib.py’ saved [2289/2289]



In [13]:
prediction= prediction.repartition(1)
prediction.write.json('a2_m3.json')

In [14]:
!zip -r a2_m3.json.zip a2_m3.json

  adding: a2_m3.json/ (stored 0%)
  adding: a2_m3.json/part-00000-42110e1c-5d7a-4e6f-82f2-94f2646374b7.json (deflated 89%)
  adding: a2_m3.json/_SUCCESS (stored 0%)
  adding: a2_m3.json/._SUCCESS.crc (stored 0%)
  adding: a2_m3.json/.part-00000-42110e1c-5d7a-4e6f-82f2-94f2646374b7.json.crc (stored 0%)


In [15]:
!base64 a2_m3.json.zip > a2_m3.json.zip.base64

In [16]:
from rklib import submit
key = "pPfm62VXEeiJOBL0dhxPkA"
part = "EOTMs"
email = "ia11287n@pace.edu"
secret = "x7nb1Htm6v4kOumN"

with open('a2_m3.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"Cu6KW","id":"f_F-qCtuEei_fRLwaVDk3g~Cu6KW~R40tJBZQEemrdBIbNUekVA","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
